In [1]:
import netgen.gui
from ngsolve import *
from ngsolve.internal import visoptions
from netgen.meshing import ImportMesh

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2304
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 10 thread(s)


2024-10-16 23:20:46.484 Python[25941:3978994] CGSSetSurfaceColorSpace failed - error 1000 (windowID:41386)
2024-10-16 23:20:46.484 Python[25941:3978994] CGSSetSurfaceColorSpace failed - error 1000 (windowID:41386)
2024-10-16 23:20:46.630 Python[25941:3978994] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-16 23:20:46.630 Python[25941:3978994] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [2]:
ngmesh = ImportMesh('cell_mesh_R1_opt.mesh')

In [3]:
from netgen.meshing import FaceDescriptor
isinpz = lambda el: all((((ngmesh[v][2] < -26)) for v in el.vertices))
fd_id = ngmesh.Add(FaceDescriptor(surfnr=1,domin=1,domout=0,bc=2))
for el in ngmesh.Elements2D():
    if isinpz(el):
        el.index = fd_id 
ngmesh.SetBCName(fd_id-1, "down")

In [4]:
mesh = Mesh(ngmesh)
Draw(mesh)
mesh.GetBoundaries()

('default', 'down')

In [5]:
#---------------- parameters ------------------
import numpy as np
Pi = np.pi
nu = 1
Pout = 0.01
Yg = CoefficientFunction(1)
Draw(Yg, mesh, "stiff")

In [6]:
orderset = 2
dimset = mesh.dim
fes = H1(mesh, order=orderset,  dirichlet="down", dim=dimset)

In [7]:
def SolveLEPDE(fes, Yg, nu, Pout):
    # TeTfunctions
    u = fes.TrialFunction()
    v = fes.TestFunction()

    # bilinear form
    lame1 = Yg / (2 * (1 + nu))
    lame2 = Yg  * nu /( (1 + nu) * ( 1 - 2 * nu))
    u_grad = grad(u).Trace()
    v_grad = grad(v).Trace()
    u_div = Trace(u_grad * Id(dimset)).Compile()
    v_div = Trace(u_grad * Id(dimset)).Compile()
    a = BilinearForm(fes, symmetric=True, check_unused=False)
    a += lame1 * InnerProduct(u_grad, v_grad) * ds
    a += lame2 * u_div * v_div * ds
    a.Assemble()

    # linear form
    nsurf = specialcf.normal(dimset)
    force = Pout * nsurf
    f = LinearForm(fes)
    f += force*v*ds
    f.Assemble()

    # initial gridfunction and solve equation
    gfu = GridFunction(fes)
    gfu.vec.data = a.mat.Inverse(fes.FreeDofs())*f.vec

    return gfu

In [8]:
gfu = GridFunction(fes)
gfu = SolveLEPDE(fes, Yg, nu, Pout)
Draw(gfu,mesh,'u')

In [9]:
u_grad = grad(gfu)
I = Id(dimset)
lame1 = Yg / (2 * (1 + nu))
lame2 = Yg  * nu /( (1 + nu) * ( 1 - 2 * nu))
# E = lame1 * (u_grad.trans + u_grad - 2 * I) + lame2 * InnerProduct((u_grad.trans - I),I ) * I
E = lame1 * (u_grad.trans + u_grad - 2 * I) + lame2 * (div(gfu) - 3)* I
gfw = GridFunction(fes)

In [16]:
RE = E.Reshape((3,3))
v = (RE.Eig()[8], RE.Eig()[4], RE.Eig()[0])

In [17]:
Draw(v,mesh,'eig')

In [2]:
help(VTKOutput)

Help on class VTKOutput in module ngsolve.comp:

class VTKOutput(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      VTKOutput
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Do(...)
 |      Do(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. Do(self: ngsolve.comp.VTKOutput, time: float = -1, vb: ngsolve.comp.VorB = <VorB.VOL: 0>) -> str
 |      
 |      
 |      Write mesh and fields to file. When called several times on the same object
 |      an index is added to the output file name. A meta file (.pvd) is written 
 |      (unless in legacy mode).
 |      
 |      Returns string of the output filename.
 |      
 |      Parameters:
 |      
 |      time : 
 |        associate a time to the current output
 |      
 |      vb: VOL_or_BND (default VOL)
 |        defines if output is done on the volume (VOL) or surface mesh (BND).
 |                  .
 |      
 |      2. Do(self: ngsolve.comp.